In [22]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [23]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [24]:
x_val = x_train[50000:60000]
x_train = x_train[0:50000]
y_val = y_train[50000:60000]
y_train = y_train[0:50000]
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) + "*" + str(x_train.shape[2]) + "image")

train data has 50000 samples
every train data is 28*28image


In [25]:
print("validation data has " + str(x_val.shape[0]) + " samples")
print("every train data is " + str(x_val.shape[1]) + "*" + str(x_val.shape[2]) + " images")

validation data has 10000 samples
every train data is 28*28 images


In [26]:
(x_train[0][8])

array([  0,   0,   0,   0,   0,   0,   0,  18, 219, 253, 253, 253, 253,
       253, 198, 182, 247, 241,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=uint8)

In [27]:
import numpy as np
x_train = np.reshape(x_train, (50000,28,28,1))
x_val = np.reshape(x_val, (10000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(50000, 28, 28, 1)
(10000, 28, 28, 1)


In [28]:
# 데이터 정규화 (데이터의 모든 값이 0~255 범위 안에 있으므로, 255로 값을 나눠 0~1 사이로 정규화)
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [29]:
# One-Hot 인코딩으로 변경 (손실함수에서 크로스 엔트로피를 계산하기 위함)
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [30]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [31]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [32]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [33]:
# 첫 번째 CONV 레이어는 총 16개의 필터를 가지고 있고, 각각 사이즈는 5 * 5
W_conv1 = weight_variable([5,5,1,16])
b_conv1 = bias_variable([16])  # 편향값은 필터 개수만큼 들어감

In [34]:
# 활성함수로 ReLU 사용
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)

In [35]:
# 풀링 레이어를 적용하여 맵의 크기 줄여줌
h_pool1 = max_pool_2x2(h_conv1)
# 28 * 28 에서 14 * 14 가 됨

In [37]:
# 두 번째 CONV 레이어는 총 32개의 필터를 가지고 있음
W_conv2 = weight_variable([5,5,16,32])
b_conv2 = bias_variable([32])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
# 두 번째 풀링 레이어 이후 액티베이션 맵의 사이즈는 7 * 7 이 됨